# Option Pricing based on Heston Model

We use Monte Carlo simulation to implement the Heston model

In [38]:
import os
import time
import pathlib
# import numba # potential speed up here
# import cupy as cp # only works with nvidia gpu
import numpy as np
import pandas as pd
from tqdm import tqdm

## 0 First steps  

We first implement a scalar version of the model.

In [81]:
# set some parameters
num_sims = 100000;   # Number of simulated asset paths
num_intervals = 1000;  # Number of intervals for the asset path to be sampled 

S_0 = 100.0;    # Initial spot price
K = 100.0;      # Strike price
r = 0.0319;     # Risk-free rate
v_0 = 0.010201; # Initial volatility 
T = 1.00;       # One year until expiry

rho = -0.7;     # Correlation of asset and volatility
kappa = 6.21;   # Mean-reversion rate
theta = 0.019;  # Long run average volatility
xi = 0.61;      # "Vol of vol"

In [99]:
def generate_heston_paths(S, T, r, kappa, theta, v_0, rho, xi, 
                          steps, num_sims):  
    '''
    Produces result for a single heston run.
    
    '''
    dt = T/steps
    # size = (num_sims, steps)
    # prices = np.zeros(size)
    # vols = np.zeros(size)
    S_t = S + np.zeros(num_sims)
    v_t = v_0 + np.zeros(num_sims)
    for t in tqdm(range(steps), colour="green"):
    # for t in range(steps):
        # [hex (#00ff00), BLACK, RED, GREEN, YELLOW, BLUE, MAGENTA, CYAN, WHITE]
        WT1 = np.random.normal(0,1,size=num_sims)
        WT2 = np.random.normal(0,1,size=num_sims)
        WT3 = rho * WT1 + np.sqrt(1-rho**2)*WT2
        # WT = np.vstack((WT1, WT3)).T

        v_t = np.maximum(v_t, 0)
        S_t = S_t*(np.exp( (r- 0.5*v_t)*dt+ np.sqrt(v_t * dt) * WT1 )) #WT[:,0]
        v_t = v_t + kappa*(theta-v_t)*dt + xi*np.sqrt(v_t * dt)*WT3    #WT[:,1]
        # prices[:, t] = S_t # can be returned when plotting is required
        # vols[:, t] = v_t   # omitted to save memory 
    
    S_call = np.exp(-1 * r * T) * np.sum(np.maximum(S_t - K, 0)) / num_sims
    S_put = np.exp(-1 * r * T) * np.sum(np.maximum(K-S_t, 0)) / num_sims
    nu_avg = np.mean(v_t)
    
    return [S_call, S_put, nu_avg]

In [100]:
result = generate_heston_paths(S_0, T, r, kappa, theta, v_0,
                      rho, xi, num_intervals, num_sims)

100%|██████████| 1000/1000 [00:06<00:00, 145.91it/s]


In [101]:
for x in result:
  print(x)

6.832665687937477
3.724997727734947
0.019326318361337597


## 1 Vectorization  

To produce a large number of results, we need to utilize the built-in vectorization in `numpy`.

We first load data for experiment and preprocess (roughly)

In [104]:
parent_path = str(pathlib.Path(os.getcwd()).parent)
df = pd.read_csv(os.path.join(parent_path, 'data/roughwork.csv'))
df.head()

,Unnamed: 0,optionid,securityid,strike,expiration,callput,contractsize,date_traded,contract_price,asset_price,contract_volume,days_to_maturity,rate,volatility
0,0,164100236.0,701057.0,22500.0,2020-03-20,C,2.0,2020-03-13,7.0,15938.0,152.0,7.0,0.858947,0.78887
1,1,164100240.0,701057.0,24500.0,2020-03-20,C,2.0,2020-03-13,22.0,15938.0,31.0,7.0,0.858947,0.78887
2,2,165303462.0,701057.0,22750.0,2020-03-20,C,2.0,2020-03-13,10.0,15938.0,7.0,7.0,0.858947,0.78887
3,3,165303464.0,701057.0,23750.0,2020-03-20,C,2.0,2020-03-13,6.0,15938.0,14.0,7.0,0.858947,0.78887
4,4,165303465.0,701057.0,24250.0,2020-03-20,C,2.0,2020-03-13,7.0,15938.0,3.0,7.0,0.858947,0.78887


Here we use (almost) the same approach to model average long run volatility $\theta$, mean reversion rate of volatility $\kappa$, and the variance of volatility $\xi$.  

**Question:** Is this the best approach? Are there better approaches? Perhaps options with the same underlying assest should take correlated values?

In [105]:
# divide by contract size for single share
df.strike = df.strike / (df.contractsize*1000)
df.asset_price = df.asset_price / (df.contractsize*1000)
df.contract_price = df.contract_price / df.contractsize

# drop contract volume
df.drop(['contract_volume'], axis = 1)

# drop small strike prices
df = df.drop(df[df.strike<0.1].index)


# add moneyness
df['moneyness'] = df.strike / df.asset_price

# We may  add the following to the function for heston simulation

# add average long run volatility (theta)
df['mean_volatility'] = 0.001 + 0.05 * np.random.rand(len(df))

# add mean reversion rate of volatility (kappa)
df['reversion'] = 0.01 + 5 * np.random.rand(len(df))

# add variance of volatility
df['var_of_vol'] = 0.01 + 0.7 * np.random.rand(len(df))

# add 
df.head()

,Unnamed: 0,optionid,securityid,strike,expiration,callput,contractsize,date_traded,contract_price,asset_price,contract_volume,days_to_maturity,rate,volatility,moneyness,mean_volatility,reversion,var_of_vol
0,0,164100236.0,701057.0,11.250,2020-03-20,C,2.0,2020-03-13,3.5,7.969,152.0,7.0,0.858947,0.78887,1.411720,0.027608,4.898995,0.579251
1,1,164100240.0,701057.0,12.250,2020-03-20,C,2.0,2020-03-13,11.0,7.969,31.0,7.0,0.858947,0.78887,1.537207,0.005500,0.280256,0.591795
2,2,165303462.0,701057.0,11.375,2020-03-20,C,2.0,2020-03-13,5.0,7.969,7.0,7.0,0.858947,0.78887,1.427406,0.016258,4.320948,0.320596
3,3,165303464.0,701057.0,11.875,2020-03-20,C,2.0,2020-03-13,3.0,7.969,14.0,7.0,0.858947,0.78887,1.490149,0.037145,4.523911,0.571365
4,4,165303465.0,701057.0,12.125,2020-03-20,C,2.0,2020-03-13,3.5,7.969,3.0,7.0,0.858947,0.78887,1.521521,0.027753,1.550456,0.296295


In [125]:
def generate_heston_vec(df, steps, num_sims):  

    '''
    Produces result for multiple heston runs for call options only.

    Args:  
        - df: dataframe, containing all parameters
        - steps: int, num time steps
        - num_sims: int, no. of simulations for each sample  

    Output:  
        - result: dataframe, containing average prices over num_sims
    '''  

    N     = len(df)
    # out   = np.zeros((N, ))
    dt    = df['days_to_maturity'].values /steps + np.zeros((num_sims, N))
    S_t   = df['asset_price'].values + np.zeros((num_sims, N))
    v_t   = df['volatility'].values + np.zeros((num_sims, N))
    r     = df['rate'].values + np.zeros((num_sims, N))
    theta = df['mean_volatility'].values + np.zeros((num_sims, N))
    kappa = df['reversion'].values + np.zeros((num_sims, N))
    xi    = df['var_of_vol'].values + np.zeros((num_sims, N))
    K     = df['strike'].values + np.zeros((num_sims, N))
    
    # for t in tqdm(range(steps), colour="green"):
        
    #     # the random normal samples are of shape (num_sims, N)
    #     WT1 = np.random.normal(0,1,size=(num_sims, N))
    #     WT2 = np.random.normal(0,1,size=(num_sims, N))
    #     WT3 = rho * WT1 + np.sqrt(1-rho**2)*WT2

    #     v_t = np.maximum(v_t, 0)
    #     S_t = S_t*(np.exp( (r- 0.5*v_t)*dt+ np.sqrt(v_t * dt) * WT1 )) 
    #     v_t = v_t + kappa*(theta-v_t)*dt + xi*np.sqrt(v_t * dt)*WT3
    
    S_call = np.exp(-1 * r * T) * np.sum(np.maximum(S_t - K, 0), axis = 0) / num_sims
    # S_put = np.exp(-1 * r * T) * np.sum(np.maximum(K-S_t, 0)) / num_sims
    nu_avg = np.mean(v_t)
    
    return S_call

In [126]:
generate_heston_vec(df, num_intervals, num_sims)

KeyboardInterrupt: 